In [25]:
#Data Mining Assignment #2
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
class FeedBack: 
    def __init__(self, type, array):
        self.type = type
        self.array = array
class Diction:
    def __init__(self, dictWord, value):
        self.dictWord = dictWord
        self.value = value

In [26]:
#Setting up the data
#Each row is an object with attributes Type (class), New_Sentence, Sentence_id
file = pd.read_csv('train_data.csv')
numRows = len(file)
numTrain = round(numRows*.6) #60% 36069
numValid = round(numRows*.2) #20% 12023
numTest = round(numRows*.2) #20% 12023
train = [] #60%
validation = [] #20%
test = [] #20%
count = 0
for index, row in file.iterrows():
    if index == numTrain:
        break
    if type(row.New_Sentence) != float:
        train.append(row)
for index, row in file.iterrows():
    if index < numTrain:
        continue
    if index == numTrain + numValid:
        break 
    if type(row.New_Sentence) != float:
        validation.append(row)
for index, row in file.iterrows():
    if index < numTrain + numValid:
        continue
    if type(row.New_Sentence) != float:
        test.append(row)
#the data is missing sentences for the attributes so throw them away

In [27]:
Dictionary = []
for each in train:
    sentence = re.split('[\\s|.|,|//|?|!|;|:|"]',each.New_Sentence)
    for word in sentence:
        if word == '':
            continue
        match = False
        for dict in Dictionary:
            if dict.dictWord == word.lower():
                match = True
                dict.value += 1
                break
        if not match: 
            Dictionary.append(Diction(word.lower(),1))
tempIndices = []
for index in range(len(Dictionary)):
    if Dictionary[index].value < 5:
        tempIndices.append(index)
tempTrack = 0
for index in tempIndices:
    Dictionary.pop(index - tempTrack)
    tempTrack += 1
for words in Dictionary:
    if words.value < 5:
        print(words.dictWord , words.value)

In [28]:
Results = []
Responsibility = 0
Requirement = 0
Skill = 0
SoftSkill = 0
Education = 0
Experience = 0
#Had to learn regex so the splitting would work properly
for each in train:
    sentence = re.split('[\\s|.|,|//|?|!|;|:|"]',each.New_Sentence)
    newType = True
    spot = 0
    for index in range(len(Results)):
        if Results[index].type == each.Type:
            newType = False
            spot = index
            break
    if newType:
        feedBack = FeedBack(each.Type, [])
    else:
        feedBack = Results[spot]
    for word in sentence:
        if word == '':
            continue
        match = False
        for dict in feedBack.array:
            if dict.dictWord == word.lower():
                dict.value = dict.value + 1
                match = True
                break
        if not match:
            feedBack.array.append(Diction(word.lower(), 1))
    if newType:
        Results.append(feedBack)
    else:
        Results[spot] = feedBack

In [29]:
for DictWord in Dictionary:
    for each in Results:
        match = False
        for words in each.array:
            if DictWord == words.dictWord:
                match = True
                break
        if not match:
            each.array.append(Diction(DictWord.dictWord, 0))

In [30]:
#Calculate the probability for "the" in all documents
theCount = 0
TotalWords = 0
for each in Results:
    for word in each.array:
        if word.dictWord == "the":
            theCount += word.value
        else:
            TotalWords += word.value
TotalWords += theCount
print('"the" count:',theCount,'Probability: ', theCount/TotalWords)

"the" count: 8095 Probability:  0.018400612823374574


In [31]:
#Calculating the probability of "the" given the document type was "Requirement"
RTheCount = 0
RTypeCount = 0
NotTheCount = 0
for each in Results:
    if(each.type == "Responsibility"): #P(Responsibility|"the")
        RTypeCount == RTypeCount + 1
        for word in each.array:
            if word.dictWord != "the":
                NotTheCount += word.value
            else:
                RTheCount += word.value
print('P("the") or P(H): ', theCount/TotalWords)
print('P(Responsibility|"the") or P(E|H) or P(E and H)/P(H): ', (RTheCount/TotalWords)/(theCount/TotalWords)) 
print('P(Responsibility|not "the") or P(E|not H) or P(E and not H)/P(not H): ', (NotTheCount/TotalWords)/((TotalWords - theCount)/TotalWords)) 
print('P(E) or P(E|H)P(H) + P(E|notH)P(notH) : ',  ((RTheCount/TotalWords)/(theCount/TotalWords))*(theCount/TotalWords) + ((NotTheCount/TotalWords)/((TotalWords - theCount)/TotalWords))*((TotalWords - theCount)/TotalWords))
print('P("the"|Responsibility) : ', ((theCount/TotalWords)*((RTheCount/TotalWords)/(theCount/TotalWords)))/(((RTheCount/TotalWords)/(theCount/TotalWords))*(theCount/TotalWords) + ((NotTheCount/TotalWords)/((TotalWords - theCount)/TotalWords))*((TotalWords - theCount)/TotalWords)))

P("the") or P(H):  0.018400612823374574
P(Responsibility|"the") or P(E|H) or P(E and H)/P(H):  0.5843113032736257
P(Responsibility|not "the") or P(E|not H) or P(E and not H)/P(not H):  0.31785677896238385
P(E) or P(E|H)P(H) + P(E|notH)P(notH) :  0.32275970549927147
P("the"|Responsibility) :  0.033311735872443515


In [68]:
def probs(word, sentenceType): #returns word given sentenceType for example Requirement 
    wordNotCount = 0 #P(notWord)
    wordTypeCount = 0 #P(word|type)
    typeCount = 0 #type Count
    wordCount = 0 #P(word)
    for each in Results:
        if(each.type == sentenceType):
            typeCount == typeCount + 1
            for thisWord in each.array:
                if type(thisWord.dictWord) == Diction:
                    continue
                if thisWord.dictWord != word:
                    wordNotCount += thisWord.value
                else:
                    wordTypeCount += thisWord.value
                    wordCount += thisWord.value
        else:
            for thisWord in each.array:
                if type(thisWord.dictWord) == Diction:
                    continue
                if thisWord.dictWord != word:
                    wordNotCount += thisWord.value
                else:
                    wordCount += thisWord.value
    if wordCount == 0:
        return 0.0
    
    Hypo = wordCount/TotalWords #P(word)
    EGivenHypo = (wordTypeCount/TotalWords)/(Hypo) #P(Type|word)
    EGivenNotHypo = (wordNotCount/TotalWords)/((TotalWords-wordCount)/TotalWords)
    Evidence = (EGivenHypo*Hypo) + (EGivenNotHypo*((TotalWords-wordCount)/TotalWords))
    HGivenEvidence = (Hypo*EGivenHypo)/(Evidence)
    return HGivenEvidence
    

In [83]:
ResponsibilityProb = []
RequirementProb = []
SkillProb = []
SoftSkillProb = []
EducationProb = []
ExperienceProb = []
for each in Results:
    if each.type == "Responsibility":
        for word in Dictionary:
            if type(word.dictWord) == Diction:
                continue
            ResponsibilityProb.append([word.dictWord,probs(word.dictWord, each.type)])
    elif each.type == "Requirement":
        for word in Dictionary:
            if type(word.dictWord) == Diction:
                continue
            RequirementProb.append([word.dictWord,probs(word.dictWord, each.type)])
    elif each.type == "Skill":
        for word in Dictionary:
            if type(word.dictWord) == Diction:
                continue
            SkillProb.append([word.dictWord,probs(word.dictWord, each.type)])
    elif each.type == "SoftSkill":
        for word in Dictionary:
            if type(word.dictWord) == Diction:
                continue
            SoftSkillProb.append([word.dictWord,probs(word.dictWord, each.type)])
    elif each.type == "Education":
        for word in Dictionary:
            if type(word.dictWord) == Diction:
                continue
            EducationProb.append([word.dictWord,probs(word.dictWord, each.type)])
    else:
        for word in Dictionary:
            if type(word.dictWord) == Diction:
                continue
            ExperienceProb.append([word.dictWord,probs(word.dictWord, each.type)])
    

In [88]:
def sortBy(array):
    return array[1]
ResponsibilityProb.sort(reverse=True,key=sortBy)
RequirementProb.sort(reverse=True,key=sortBy)
SkillProb.sort(reverse=True,key=sortBy)
SoftSkillProb.sort(reverse=True,key=sortBy)
EducationProb.sort(reverse=True,key=sortBy)
ExperienceProb.sort(reverse=True,key=sortBy)

In [89]:
ResponsibilityProb = ResponsibilityProb[0:10]
RequirementProb = RequirementProb[0:10]
SkillProb = SkillProb[0:10]
SoftSkillProb = SoftSkillProb[0:10]
EducationProb = EducationProb[0:10]
ExperienceProb = ExperienceProb[0:10]

In [90]:
print("Responsibility")
for each in ResponsibilityProb:
    print(each)
print("Requirement")
for each in RequirementProb:
    print(each)
print("Skill")
for each in SkillProb:
    print(each)
print("SoftSkill")
for each in SoftSkillProb:
    print(each)
print("Education")
for each in EducationProb:
    print(each)
print("Experience")
for each in ExperienceProb:
    print(each)

Responsibility
['and', 0.023784478315813255]
['the', 0.01083455880668673]
['to', 0.009682363902445791]
['of', 0.007641399538685021]
['with', 0.0049943549073216045]
['for', 0.0049115286391827806]
['in', 0.004605703450327186]
['a', 0.0029260718977666307]
['business', 0.0024873521565799595]
['will', 0.0022844712836953824]
Requirement
['and', 0.014053576139580318]
['to', 0.008650995512498583]
['in', 0.006328262372356072]
['of', 0.005548385596521213]
['experience', 0.005482770417078876]
['with', 0.004137927860129753]
['the', 0.003629366256815156]
['a', 0.0035325631626425135]
['ability', 0.003284641563489384]
['or', 0.002810188310167647]
Skill
['and', 0.004874468054482736]
['experience', 0.003190210201722296]
['in', 0.0025985517032621605]
['of', 0.002523726998980713]
['with', 0.0016703465046153704]
['knowledge', 0.0014823396336568636]
['to', 0.0011565896461908328]
['a', 0.001028605353810327]
['or', 0.0008327658869646832]
['is', 0.0008129301565118902]
SoftSkill
['and', 0.00990018664286294]
['